Predicting rental durations by experimenting with regression models. The goal is to achieve a test set MSE of 3 or lower for enhanced inventory planning efficiency.

The data provided is in the csv file `rental_info.csv`. It has the following features:
- `"rental_date"`: The date (and time) the customer rents the DVD.
- `"return_date"`: The date (and time) the customer returns the DVD.
- `"amount"`: The amount paid by the customer for renting the DVD.
- `"amount_2"`: The square of `"amount"`.
- `"rental_rate"`: The rate at which the DVD is rented for.
- `"rental_rate_2"`: The square of `"rental_rate"`.
- `"release_year"`: The year the movie being rented was released.
- `"length"`: Lenght of the movie being rented, in minuites.
- `"length_2"`: The square of `"length"`.
- `"replacement_cost"`: The amount it will cost the company to replace the DVD.
- `"special_features"`: Any special features, for example trailers/deleted scenes that the DVD also has.
- `"NC-17"`, `"PG"`, `"PG-13"`, `"R"`: These columns are dummy variables of the rating of the movie. It takes the value 1 if the move is rated as the column name and 0 otherwise. For your convinience, the reference dummy has already been dropped.

In [15]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV



df = pd.read_csv("rental_info.csv")
df["return_date"] = pd.to_datetime(df["return_date"])
df["rental_date"] = pd.to_datetime(df["rental_date"])
df['rental_length'] = df["rental_date"] - df["return_date"]
df['rental_length_days'] = df['rental_length'].dt.days



df["deleted_scenes"] =  np.where(df["special_features"].str.contains("Deleted Scenes"), 1,0)
df["behind_the_scenes"] =  np.where(df["special_features"].str.contains("Behind the Scenes"), 1, 0)


cols_to_drop = ["special_features", "rental_length", "rental_length_days", "rental_date", "return_date"]

X = df.drop(cols_to_drop, axis=1)
y = df['rental_length_days']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=9)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=9)

lasso_model = Lasso(random_state=9, alpha=0.01) 

lasso_model.fit(X_train, y_train)


lasso_coef = lasso_model.coef_

Xl_train = X_train.iloc[:, lasso_coef > 0] 
Xl_test = X_test.iloc[:, lasso_coef > 0]



lr_model = LinearRegression().fit(Xl_train, y_train) #fit
y_pred = lr_model.predict(Xl_test)    
mse = mean_squared_error(y_test, y_pred)        #evaluate



param_dist = {'n_estimators': np.arange(1,101,1), 'max_depth':np.arange(1,11,1)}


rf = RandomForestRegressor()

rsearch = RandomizedSearchCV(rf, param_distributions=param_dist, cv=5, random_state=5)

rsearch.fit(X_train, y_train)

hyperp = rsearch.best_params_

rf= RandomForestRegressor(n_estimators=hyperp['n_estimators'], max_depth=hyperp['max_depth'], random_state=9)

rf.fit(X_train,y_train)
rf_pred = rf.predict(X_test)
mse_random_forest = mean_squared_error(y_test, rf_pred)

best_model = rf
best_mse = mse_random_forest
best_mse

2.217807141018225